In [ ]:
from urllib3 import PoolManager
from lxml import etree

#opener = request.build_opener()
#tree = etree.parse(opener.open(url))
#tree = etree.parse(data)
#data = tree.opener.open(url)
http = PoolManager()
def get_source_page(url):
    http = PoolManager()
    response = http.request('GET',url)
    source_page = response.read()
    return source_page

In [ ]:
url = 'http://www.thegoodscentscompany.com/fragonly-a.html'
http = PoolManager()
parser = etree.HTMLParser()
alfa_urls = []
with http.urlopen('GET',url, preload_content=False) as f:
     tree = etree.parse(f, parser)
for x in tree.getroot().xpath("//div[@class='alphaENIC1']"):#.find_class('alphaENIC1')
    #strx = etree.tostring(x,method='html')
    for y in x.getiterator():
        #print(y.attrib)
        if y.attrib.has_key('href'):
            #print(y.attrib.get('href'))
            alfa_urls.append(y.attrib.get('href'))
            
    #etree.XML(strx)
    
    
    


In [ ]:
from collections import OrderedDict
alfa_urls = list(OrderedDict.fromkeys(alfa_urls))

In [ ]:
alfa_urls

In [ ]:
#print(etree.tostring(tree.getroot(), pretty_print=True, method='xml'))
from bs4 import BeautifulSoup
soup = BeautifulSoup(etree.tostring(tree.getroot(), pretty_print=True),'lxml')
soup

In [10]:
for url in alfa_urls:
    print(url);
#source_page = get_source_page('http://www.thegoodscentscompany.com/fragonly-a.html')

http://www.thegoodscentscompany.com/fragonly-a.html
http://www.thegoodscentscompany.com/fragonly-b.html
http://www.thegoodscentscompany.com/fragonly-c.html
http://www.thegoodscentscompany.com/fragonly-d.html
http://www.thegoodscentscompany.com/fragonly-e.html
http://www.thegoodscentscompany.com/fragonly-f.html
http://www.thegoodscentscompany.com/fragonly-g.html
http://www.thegoodscentscompany.com/fragonly-h.html
http://www.thegoodscentscompany.com/fragonly-i.html
http://www.thegoodscentscompany.com/fragonly-jk.html
http://www.thegoodscentscompany.com/fragonly-l.html
http://www.thegoodscentscompany.com/fragonly-m.html
http://www.thegoodscentscompany.com/fragonly-n.html
http://www.thegoodscentscompany.com/fragonly-o.html
http://www.thegoodscentscompany.com/fragonly-p.html
http://www.thegoodscentscompany.com/fragonly-q.html
http://www.thegoodscentscompany.com/fragonly-r.html
http://www.thegoodscentscompany.com/fragonly-s.html
http://www.thegoodscentscompany.com/fragonly-t.html
http://www.

In [11]:
def get_xhtml(url,pool):
    #http = PoolManager()
    #response = pool.request('GET',url)
    with pool.urlopen('GET',url, preload_content=False) as f:
        tree = etree.parse(f, parser)
    return tree

In [ ]:
url = 'http://www.thegoodscentscompany.com/fragonly-b.html'
http = PoolManager()
xhtml = get_xhtml(url,http)
#source_page = response.read()
#data = str(response.data)

In [ ]:
component_table = xhtml.xpath("/html//table[1]")[0]
for c in component_table.getiterator():
    if c.attrib.has_key('href'):
        print(c.values()[0])
        
    

In [12]:
alfa_urls[0]

'http://www.thegoodscentscompany.com/fragonly-a.html'

In [ ]:
c_url = 'http://www.thegoodscentscompany.com/data/rw1594981.html'
c_xhtml = get_xhtml(c_url,http)

In [ ]:
c_xhtml.find(".//title").text

In [ ]:
print(c_xhtml.getroot)

In [13]:
def clean_string(txt):
    txt = txt.replace('\xa0',' ').replace('Â\xa0',' ').replace('Â','')
    return txt

In [14]:
def get_sub_before_first_comma(txt):
        txt = txt + ','
        l = txt[0:txt.find(',')].strip()
        try:
            l.remove("")
        except: None
        return l

In [15]:
def split_html_string(txt):
    odor_descr_str = txt.replace('Â\xa0Â\xa0',',').replace('Â\xa0Â','').replace('Â','')
    odor_descr_list = odor_descr_str.split(',')
    try:
        odor_descr_list.remove("")
    except ValueError: None
    return odor_descr_list
    

In [16]:
def get_substantivity(txt):
    txt = txt[0:txt.lower().find('hour(s)')].strip()
    return txt

In [40]:
import re
def get_numeric_value(txt):
    txt = re.findall(r'[0-9.]',txt)
    txt = ''.join(txt)
    return ''.join(txt)
    

In [18]:
def get_filename(url,folder):
    return folder + '/' + url[url.rfind('/')+1:url.rfind('.')] + '.json'

In [36]:
def get_min_value(txt):
    txt = txt.replace('Â','')
    txt = txt.split(' ')
    txt = txt[0].replace(' to','').replace('>','').replace('<','')
    return txt  

In [37]:
def get_flash_point_celsium(txt):
    txt = txt.split('Â')
    txt = txt[4].replace('(','').replace(' ','').replace('>','').replace('<','')
    return txt
    

In [38]:
def get_logP(txt):
    txt = txt.replace('(est)','').replace(' ','')
    return txt

In [44]:
import pandas as pd
import unidecode
import json
import numpy as np
datafolder = 'tgsc1'
http = PoolManager()
tgsc_component = {}
for url in alfa_urls:
    print(url)

    #tgsc_component = {}
    xhtml = get_xhtml(url,http)
    component_table = xhtml.xpath("/html//table[1]")[0]
    for component in component_table.getiterator():
        #print('    ' + str(component))
       
        if component.attrib.has_key('href'):
            #print('    ' + component.values()[0])
            c_url = component.values()[0]
            c_xhtml = get_xhtml(c_url,http)
            component_dict = {}
            name = c_xhtml.find(".//title").text
            component_dict['name'] = clean_string(name) #.replace('\xa0',' ').replace('Â\xa0',' ')
            for section in c_xhtml.xpath('//table[@id="chemdata"]'):
                #print(section)
                df_main = pd.DataFrame()
                
                try:
                    df_main = pd.read_html(io=etree.tostring(section, encoding='utf-8'))[0]
                except: 
                    print('    ' + c_url)
                    raise Exception("Convert to Pandas Error")
                    
                df_main.columns = ['property', 'value']
                try:
                    iupac = df_main[df_main["property"] == "IUPAC Name:"]["value"].values[0]
                    component_dict['iupac'] = clean_string(iupac) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
                try:
                    full_name = df_main[df_main["property"] == "Name:"]["value"].values[0]
                    component_dict['full_name'] = clean_string(full_name) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
                try:
                    cas = df_main[df_main["property"] == "CAS Number:"]["value"].values[0]
                    component_dict['cas'] = clean_string(cas) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
            sectionclass = c_xhtml.xpath("//div[@class='sectionclass']")
            for s in sectionclass:
                #print(s.text)
                if s.text == "Physical Properties: ":
                    #print(s.getnext())
                    df_phy = pd.DataFrame()
                    df_phy = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                    if len(df_phy) > 1:
                        df_phy.columns = ['property', 'value']
                        try:
                            specific_gravity = df_phy[df_phy["property"] == "Specific Gravity:"]["value"].values[0]
                            #print("specific_gravity=" + specific_gravity)
                            #print("specific_gravity=" + get_min_value(specific_gravity))
                            component_dict['specific_gravity'] = get_min_value(clean_string(specific_gravity)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            boiling_point = df_phy[df_phy["property"] == "Boiling Point:"]["value"].values[0]
                            #print("boiling_point=" + boiling_point)
                            #print("boiling_point=" + get_min_value(clean_string(boiling_point)))
                            component_dict['boiling_point'] = get_min_value(clean_string(boiling_point)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            refractive_index = df_phy[df_phy["property"] == "Refractive Index:"]["value"].values[0]
                            #print("refractive_index=" + refractive_index)
                            #print("refractive_index=" + get_min_value(refractive_index))
                            component_dict['refractive_index'] = get_min_value(clean_string(refractive_index)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            flash_point = df_phy[df_phy["property"] == "Flash Point:"]["value"].values[0]
                            #print("flash_point=" + flash_point)
                            #print("flash_point=" + get_numeric_value(get_flash_point_celsium(flash_point)))
                            if str(flash_point) != 'nan':
                                component_dict['flash_point'] = get_numeric_value(get_flash_point_celsium(flash_point)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            logP = df_phy[df_phy["property"] == "logP (o/w):"]["value"].values[0]
                            #print("logP=" + logP)
                            #print("logP=" + get_logP(logP))
                            component_dict['logP'] = get_logP(clean_string(logP)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
    
                if s.text == "Organoleptic Properties: ":
                    #print(s.getnext())
                    df_org = pd.DataFrame()
                    df_org = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                    if len(df_org) > 1:
                        df_org.columns = ['property', 'value']
                        try:
                            odor_type = df_org[df_org["property"] == "Odor Type:"]["value"].values[0]
                            #print(odor_type)
                            component_dict['odor_type'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            #odor_type = df[df["property"] == "Odor Type:"]["value"].values[0]
                            odor_strength = df_org[df_org["property"] == "Odor Strength:"]["value"].values[0]
                            #print(odor_strength)
                            #print(clean_string(get_sub_before_first_comma(odor_strength)))
                            component_dict['odor_strength'] = clean_string(get_sub_before_first_comma(odor_strength)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                            #component_dict['odor_strength'] = odor_strength
                        except IndexError: None
                        try:
                            odor_description = split_html_string(df_org['value'][2])
                            if len(odor_description) > 0:
                                component_dict['odor_description'] = odor_description
                        except IndexError: None
                        except KeyError: print('    ' + c_url)
                        try:
                            odor_substantivity = df_org[df_org["property"] == "Substantivity:"]["value"].values[0]
                            try:
                                odor_substantivity = get_numeric_value(clean_string(odor_substantivity))
                                component_dict['odor_substantivity'] = get_substantivity(odor_substantivity)

                            except AttributeError: None
                        except IndexError: None
            tgsc_component[c_url] = component_dict
            #filename = get_filename(url,datafolder)
filename = "/home/masha/scent_datasets/tgsc1/fragonly-All.json"
with open(filename, 'w') as outfile:
    json.dump(tgsc_component, outfile)
                #print section.xpath('h1[1]/text()')[0],
                #print section.xpath('h3[1]/text()')[0]


http://www.thegoodscentscompany.com/fragonly-a.html
    http://www.thegoodscentscompany.com/data/co1067571.html
    http://www.thegoodscentscompany.com/data/rw1537431.html
    http://www.thegoodscentscompany.com/data/rw1003241.html
    http://www.thegoodscentscompany.com/data/rw1004091.html
    http://www.thegoodscentscompany.com/data/es1052621.html
    http://www.thegoodscentscompany.com/data/ab1657301.html
    http://www.thegoodscentscompany.com/data/co1657731.html
    http://www.thegoodscentscompany.com/data/rw1056571.html
    http://www.thegoodscentscompany.com/data/ab1092781.html
http://www.thegoodscentscompany.com/fragonly-b.html
    http://www.thegoodscentscompany.com/data/rw1464061.html
    http://www.thegoodscentscompany.com/data/rw1464271.html
    http://www.thegoodscentscompany.com/data/rw1412231.html
    http://www.thegoodscentscompany.com/data/rw1122951.html
    http://www.thegoodscentscompany.com/data/rw1123231.html
    http://www.thegoodscentscompany.com/data/ex1022771.h

In [26]:
df_phy.head(10)
#c_url

,property,value
0,Appearance:,colorless clear liquid (est)
1,Assay:,96.00Â toÂ 100.00Â %Â
2,Food Chemicals Codex Listed:,No
3,Specific Gravity:,0.94500Â toÂ 0.96500Â @Â 25.00 Â°C.
4,Pounds per Gallon - (est).:,7.863Â toÂ 8.030
5,Refractive Index:,1.45500Â toÂ 1.49900Â @Â 20.00 Â°C.
6,Boiling Point:,60.00Â toÂ 63.00Â Â°C. @ 1.00Â mm Hg
7,Acid Value:,1.00Â max.Â KOH/g
8,Vapor Pressure:,0.027000 mm/Hg @ 25.00 Â°C. (est)
9,Flash Point:,209.00Â Â°F.Â TCCÂ ( 98.33 Â°C. )


In [ ]:
flash_point = df_phy[df_phy["property"] == "Flash Point:"]["value"].values[0]

In [ ]:
if str(flash_point) == 'nan':
    print('is nan')

In [27]:
tgsc_component

{'http://www.thegoodscentscompany.com/data/ab1000651.html': {'cas': '68917-51-1',
  'flash_point': '\xa0>100.00',
  'full_name': 'fucus vesiculosus absolute',
  'name': 'green algae absolute   68917-51-1',
  'odor_description': ['fresh',
   'marine',
   'green',
   'sandy',
   'oily',
   'painty',
   'oakmoss'],
  'odor_strength': 'medium',
  'odor_type': 'marine'},
 'http://www.thegoodscentscompany.com/data/ab1054931.html': {'cas': '8006-77-7',
  'full_name': 'pimenta officinalis lindl. fruit absolute',
  'name': 'allspice berry absolute   8006-77-7',
  'odor_description': ['sweet', 'warm', 'spicy', 'woody', 'clove'],
  'odor_strength': 'medium',
  'odor_type': 'spicy'},
 'http://www.thegoodscentscompany.com/data/ab1067541.html': {'cas': '85480-33-7',
  'full_name': 'aglaia odorata absolute',
  'name': 'aglaia odorata absolute   85480-33-7',
  'refractive_index': '1.48700',
  'specific_gravity': '0.87000'},
 'http://www.thegoodscentscompany.com/data/ab1092731.html': {'cas': '84082-36-

In [ ]:
#tgc_component
dict = {} #{1:['q','a','z']}
dict[1] = ['q','a','z']
dict

In [ ]:
txt = 'low, 123,'
txt = txt + ','
txt[0:txt.find(',')].strip()

In [ ]:

import pandas as pd
import unidecode
import json
#datafolder = 'tgsc1'
def get_component_pd(url):
    http = PoolManager()
    c_url = url
    c_xhtml = get_xhtml(c_url,http)
    component_dict = {}
    name = c_xhtml.find(".//title").text
    component_dict['name'] = clean_string(name) #.replace('\xa0',' ').replace('Â\xa0',' ')
    for section in c_xhtml.xpath('//table[@id="chemdata"]'):
        #print(section)
        df = pd.DataFrame()
        try:
            df = pd.read_html(io=etree.tostring(section, encoding='utf-8'))[0]
        except: 
            print('    ' + c_url)
            raise Exception("Convert to Pandas Error")
        df.columns = ['property', 'value']
        try:
            iupac = df[df["property"] == "IUPAC Name:"]["value"].values[0]
            component_dict['iupac'] = clean_string(iupac) #.replace('\xa0',' ').replace('Â\xa0',' ')
        except IndexError: None
        try:
            full_name = df[df["property"] == "Name:"]["value"].values[0]
            component_dict['full_name'] = clean_string(full_name) #.replace('\xa0',' ').replace('Â\xa0',' ')
        except IndexError: None
        try:
            cas = df[df["property"] == "CAS Number:"]["value"].values[0]
            component_dict['cas'] = clean_string(cas) #.replace('\xa0',' ').replace('Â\xa0',' ')
        except IndexError: None
        sectionclass = c_xhtml.xpath("//div[@class='sectionclass']")
        for s in sectionclass:
            #print(s.text)
            if s.text == "Physical Properties: ":
                #print(s.getnext())
                df_phy = pd.DataFrame()
                df_phy = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                if len(df) > 1:
                    df_phy.columns = ['property', 'value']
                    try:
                        odor_type = df[df["property"] == "Specific Gravity: "]["value"].values[0]
                        #print(odor_type)
                        component_dict['specific_gravity'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                    except IndexError: None
                    try:
                        odor_type = df[df["property"] == "Boiling Point: "]["value"].values[0]
                        #print(odor_type)
                        component_dict['boiling_point'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                    except IndexError: None
                    try:
                        odor_type = df[df["property"] == "Refractive Index: "]["value"].values[0]
                        #print(odor_type)
                        component_dict['refractive_index'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                    except IndexError: None
                        try:
                            odor_type = df[df["property"] == "Flash Point: "]["value"].values[0]
                            #print(odor_type)
                            component_dict['flash_point'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
    
                if s.text == "Organoleptic Properties: ":
                    #print(s.getnext())
                    df_org = pd.DataFrame()
                    df_org = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                    if len(df_org) > 1:
                        df_org.columns = ['property', 'value']
                        try:
                            odor_type = df_org[df_org["property"] == "Odor Type:"]["value"].values[0]
                            #print(odor_type)
                            component_dict['odor_type'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            #odor_type = df[df["property"] == "Odor Type:"]["value"].values[0]
                            odor_strength = df_org[df_org["property"] == "Odor Strength:"]["value"].values[0]
                            #print(odor_strength)
                            #component_dict['odor_strength'] = clean_string(get_sub_before_first_comma(odor_strength)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                            component_dict['odor_strength'] = odor_strength
                        except IndexError: None
                        try:
                            odor_description = split_html_string(df_org['value'][2])
                            if len(odor_description) > 0:
                                component_dict['odor_description'] = odor_description
                        except IndexError: None
                        except KeyError: print('    ' + c_url)
                        try:
                            odor_substantivity = df_org[df_org["property"] == "Substantivity:"]["value"].values[0]
                            try:
                                odor_substantivity = get_numeric_value(clean_string(odor_substantivity))
                                component_dict['odor_substantivity'] = get_substantivity(odor_substantivity)

                            except AttributeError: None
                        except IndexError: None
            tgsc_component[c_url] = component_dict
           

In [ ]:
etree.tostring(section, encoding='utf-8')
#for s in section.getiterator():
#    print(s.values())

In [ ]:
tgc_component

In [ ]:
etree.tostring(section)

In [ ]:
df1 = df["property"] == "Name:"

In [ ]:
mdl = df[df["property"] == "MDL:"]["value"].values[0]

In [ ]:
type(mdl)

In [ ]:
df.rename(index=str, columns={"1": "c1", "2": "c2"})

In [ ]:
df.head()

In [ ]:
#etree.tostring(c_xhtml)
soup = BeautifulSoup(etree.tostring(c_xhtml.getroot(), pretty_print=True),'lxml')
soup

In [ ]:
for x in tree.getroot().xpath("//div[@class='alphaENIC1']"):#.find_class('alphaENIC1')
    #strx = etree.tostring(x,method='html')
    for y in x.getiterator():
        print(y.attrib)
        if y.attrib.has_key('href'):
            #print(y.attrib.get('href'))
            alfa_urls.append(y.attrib.get('href'))
            

In [ ]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)

    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)

    def handle_data(self, data):
        print("Encountered some data  :", data)

parser = HTMLParser()
data = parser.feed(str(response.data))


In [ ]:
data = response.data.decode('utf8')

In [ ]:
data

In [ ]:
from lxml import etree

opener = request.build_opener()
tree = etree.parse(opener.open(url))
#tree = etree.parse(data)
data = tree.opener.open(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(source_page.replace('<br />','\n'),'lxml')
#article_body = soup.find('div',{'class':'notebox'})\\
notes = soup.findAll('div',{'class':'notebox'})
#a_href_notes = notes.findAll(['a']) #for a in article_body.findAll(['a']):
#notes_img = map(lambda x: x.find(['img']), notes)
for note in notes:
     n = note.find('img').get('alt')
     print(n)
#     print(n.get_text())
#    for ref_name in a.findAll('strong'):
#            #dictionary.append(ref_name.get_text())
#            print(ref_name.get_text())
    

In [ ]:
print(notes)